In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
import warnings
warnings.filterwarnings('ignore')


Reading the Datasets

In [3]:
df=pd.read_csv("C:/Users/manis/OneDrive/Documents/Python Scripts/Projects/playground-series-s4e10/train.csv")
df_t=pd.read_csv("C:/Users/manis/OneDrive/Documents/Python Scripts/Projects/playground-series-s4e10/test.csv")


In [4]:
df

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58640,58640,34,120000,MORTGAGE,5.0,EDUCATION,D,25000,15.95,0.21,Y,10,0
58641,58641,28,28800,RENT,0.0,MEDICAL,C,10000,12.73,0.35,N,8,1
58642,58642,23,44000,RENT,7.0,EDUCATION,D,6800,16.00,0.15,N,2,1
58643,58643,22,30000,RENT,2.0,EDUCATION,A,5000,8.90,0.17,N,3,0


In [5]:
df=df.drop(columns='id')
df1=df_t.drop(columns='id')

Encoding the Categorical Columns

In [7]:
#Train data
cat_col=df.select_dtypes(include=["object"]).columns.tolist()
encoder=OneHotEncoder(sparse_output=False)
O_H_E=encoder.fit_transform(df[cat_col])
one_hot_df = pd.DataFrame(O_H_E, columns=encoder.get_feature_names_out(cat_col))
df_encoded = pd.concat([df, one_hot_df], axis=1)
df_encoded = df_encoded.drop(cat_col, axis=1)
#Test data
cat_col_t=df1.select_dtypes(include=["object"]).columns.tolist()
encoder_test=OneHotEncoder(sparse_output=False)
O_H_E_t=encoder_test.fit_transform(df1[cat_col])
one_hot_df_t = pd.DataFrame(O_H_E_t, columns=encoder_test.get_feature_names_out(cat_col_t))
df_encoded_t = pd.concat([df1, one_hot_df_t], axis=1)
df_encoded_t = df_encoded_t.drop(cat_col_t, axis=1)


In [8]:
df_encoded

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,...,loan_intent_VENTURE,loan_grade_A,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_N,cb_person_default_on_file_Y
0,37,35000,0.0,6000,11.49,0.17,14,0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,22,56000,6.0,4000,13.35,0.07,2,0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,29,28800,8.0,6000,8.90,0.21,10,0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,30,70000,14.0,12000,11.11,0.17,5,0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,22,60000,2.0,6000,6.92,0.10,3,0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58640,34,120000,5.0,25000,15.95,0.21,10,0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
58641,28,28800,0.0,10000,12.73,0.35,8,1,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
58642,23,44000,7.0,6800,16.00,0.15,2,1,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
58643,22,30000,2.0,5000,8.90,0.17,3,0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Seperating the Features and Target class

In [10]:
'''X=df_encoded.drop("loan_status",axis=1)
y=df_encoded["loan_status"]'''

'X=df_encoded.drop("loan_status",axis=1)\ny=df_encoded["loan_status"]'

In [11]:
X_res=df_encoded.drop("loan_status",axis=1)
y_res=df_encoded["loan_status"]

Using SMOTETomke to Oversample-

In [13]:
smk=SMOTETomek()
X_res,y_res=smk.fit_resample(X_res,y_res)

Using Standard Scaler to normally distribute the dataset

In [15]:
'''scaler=StandardScaler()
#Train Data
scaler.fit(X_res)
Scaled_feat=scaler.transform(X_res)'''
Scaled_feat=X_res

#Test Data
'''scaler.fit(df_encoded_t)
Scaled_feat_t=scaler.transform(df_encoded_t)'''
Scaled_feat_t=df_encoded_t

Creating the DF with the processed data

In [17]:
#Train Data
X_sc=pd.DataFrame(Scaled_feat,columns=X_res.columns)
#Test Data
X_sc_t=pd.DataFrame(Scaled_feat_t,columns=df_encoded_t.columns)

In [18]:
X_sc_t

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,...,loan_intent_VENTURE,loan_grade_A,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_N,cb_person_default_on_file_Y
0,23,69000,3.0,25000,15.76,0.36,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,26,96000,6.0,10000,12.68,0.10,4,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,26,30000,5.0,4000,17.19,0.13,2,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,33,50000,4.0,7000,8.90,0.14,7,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,26,102000,8.0,15000,16.32,0.15,4,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39093,22,31200,2.0,3000,10.37,0.10,4,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
39094,22,48000,6.0,7000,6.03,0.15,3,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
39095,51,60000,0.0,15000,7.51,0.25,25,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
39096,22,36000,4.0,14000,15.62,0.39,4,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


Train Test Split

In [20]:
X_train1,X_test1,y_train1,y_test1= train_test_split(X_sc,y_res,test_size=0.2)

Delpoying Random Forest Algorithm

In [22]:
rf_classifier = RandomForestClassifier(max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100,random_state=42)
rf_classifier.fit(X_train1, y_train1)

RandomForestClassifier(max_depth=10, random_state=42)

HyperParameter Tuning 

In [24]:
param_space = {
    'n_estimators': (10, 100),          # Number of trees in the forest
    'max_depth': (1, 10),               # Maximum depth of each tree
    'min_samples_split': (2, 20),       # Minimum samples required to split a node
    'min_samples_leaf': (1, 10)         # Minimum samples required at each leaf node
}

In [25]:
opt = BayesSearchCV(
    estimator=rf_classifier,
    search_spaces=param_space,
    n_iter=32,           # Number of iterations (evaluations)
    cv=5,                # 5-fold cross-validation
    random_state=42,
    n_jobs=-1            # Use all available CPU cores
)

In [26]:
opt.fit(X_train1,y_train1)

# Output the best hyperparameters and score
print(f"Best Hyperparameters: {opt.best_params_}")
print(f"Best Score: {opt.best_score_:.4f}")

Best Hyperparameters: OrderedDict({'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100})
Best Score: 0.9613


Making Predictions

In [28]:
# Make predictions on the train set
y_pred = rf_classifier.predict(X_test1)
# Make predictions on the test set
pre_t=rf_classifier.predict(X_sc_t)

Performance of the model

In [30]:
accuracy = accuracy_score(y_test1, y_pred)

In [31]:
confusion_matrix(y_test1,y_pred)

array([[9663,  170],
       [ 598, 9072]], dtype=int64)

In [32]:
print(classification_report(y_test1, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96      9833
           1       0.98      0.94      0.96      9670

    accuracy                           0.96     19503
   macro avg       0.96      0.96      0.96     19503
weighted avg       0.96      0.96      0.96     19503



Preparing the Submission file

In [34]:
df_pred=pd.DataFrame(pre_t)
submission = pd.concat([df_t,df_pred], axis=1)
submission.rename(columns = {0:'loan_status_pred'}, inplace = True)
submission=submission[["id","loan_status_pred"]]

In [35]:
submission.to_csv('submission_3.csv', index=False)